## Summed OS volume

One way to circumvent that several outer segments are combined in one label (without filtering) is calculating the summed OS volume per image and afterwards the average per biological replicate.

In [1]:
import numpy as np
import pandas as pd

In [2]:
# Define a file path with the measurements
path = "../../measurements/wt-postnatal-development/"

In [3]:
# Open unfiltered dataset
measurements = pd.read_csv(path + "05-a-measurements-inf-reduced.csv")
measurements

,label,age,biol_repl,image_id,maximum,mean,median,minimum,sigma,sum,...,equivalent_spherical_perimeter_rescaled,equivalent_spherical_radius_rescaled,feret_diameter_rescaled,perimeter_2d_rescaled,major_axis_length_2d_rescaled,minor_axis_length_2d_rescaled,surface_area_rescaled,bbox_volume_rescaled,convex_volume_rescaled,volume_rescaled
0,1,8,1,0,803.0,314.092437,282.884766,186.0,99.098496,261639.0,...,138.287624,1.885333,8.107236,18.460443,8.872033,2.037236,85.604886,116.461211,51.389857,28.070656
1,2,8,1,0,1845.0,620.114058,466.576172,174.0,407.194822,467566.0,...,129.399980,1.823742,6.111447,8.976538,3.444867,2.044837,43.541779,42.257627,29.149001,25.408493
2,3,8,1,0,564.0,274.831858,253.494141,154.0,91.856545,62112.0,...,57.955182,1.220513,3.995289,6.273068,2.549953,1.312964,21.829812,14.018479,9.233325,7.615808
3,4,8,1,0,540.0,285.008439,268.189453,153.0,89.769054,67547.0,...,59.820799,1.240002,4.669557,6.273068,2.766553,1.117553,23.752687,16.175168,9.570308,7.986489
4,5,8,1,0,264.0,234.200000,238.798828,202.0,20.192821,5855.0,...,13.355085,0.585895,2.261000,2.777771,1.010003,0.745937,8.389508,2.426275,1.112043,0.842457
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16695,173,24,28,83,312.0,243.290323,245.492188,179.0,26.596939,60336.0,...,61.657763,1.258897,5.297594,9.737588,4.460800,1.451807,40.056207,46.705798,12.805341,8.357170
16696,174,24,28,83,685.0,346.677288,326.101562,187.0,92.460071,424333.0,...,178.734172,2.143385,8.551879,30.066147,8.226779,5.176775,131.982553,226.856733,104.262438,41.246679
16697,175,24,28,83,609.0,337.340530,326.101562,188.0,81.507414,343750.0,...,158.174366,2.016343,8.478365,21.955740,9.303697,3.873424,102.052789,176.915902,74.136187,34.338534
16698,176,24,28,83,347.0,255.431818,252.820312,193.0,37.513655,11239.0,...,19.468019,0.707387,1.910894,4.002664,1.622372,0.810159,7.356350,3.032844,1.819706,1.482724


### Determine summed volume per image

Now from the unfiltered dataset the summed volume per image is computed.

In [4]:
# Determine summe volume
summed_volume = measurements.groupby("image_id", as_index=False)["volume_rescaled"].sum()
summed_volume

,image_id,volume_rescaled
0,0,1313.491051
1,1,570.646453
2,2,601.749954
3,3,1508.502922
4,4,768.556375
...,...,...
79,79,6357.717242
80,80,3443.423715
81,81,6947.942388
82,82,3305.193424


### Determine mean summed volume per biological replicate

Again, the mean of the biological replicate per image id will be calculated and added to the summed volume. Afterwards the mean of the summed volume per biological replicate will be computed.

In [7]:
# Determine mean of biological replicate
biol_repl = measurements.groupby("image_id", as_index=False)["biol_repl"].mean()

# Add biological replicate to dataframe
summed_volume = pd.merge(summed_volume, biol_repl, how="inner")

# Determine the mean of the summed volume per biological replicate
summed_volume_mean = summed_volume.groupby("biol_repl", as_index=False)["volume_rescaled"].mean()
summed_volume_mean.rename(columns={"volume_rescaled": "summed_volume_mean"}, inplace=True)
summed_volume_mean

,biol_repl,summed_volume_mean
0,1.0,828.629153
1,2.0,808.983063
2,3.0,672.190565
3,4.0,832.021445
4,5.0,870.516099
5,6.0,881.490501
6,7.0,1241.747441
7,8.0,1837.532801
8,9.0,2130.831284
9,10.0,925.455507


### Add mean summed volume to average dataframe

Finally, the mean summed volume will be added to the average dataframe.

In [8]:
# Load dataframe containing mean values
measurements_means = pd.read_csv(path + "07-os-numbers-added.csv")

# Add summed mean volume per biological replicate
measurements = pd.merge(measurements_means, summed_volume_mean, how="inner")
measurements

,biol_repl,label,age,maximum,mean,median,minimum,sigma,sum,variance,...,equivalent_spherical_radius_rescaled,feret_diameter_rescaled,perimeter_2d_rescaled,major_axis_length_2d_rescaled,minor_axis_length_2d_rescaled,surface_area_rescaled,bbox_volume_rescaled,convex_volume_rescaled,volume_rescaled,summed_volume_mean
0,1.0,83.000000,8.0,572.072487,276.999939,253.902252,153.514021,89.207799,89571.470370,12483.492986,...,1.135134,3.933802,6.961996,2.731891,1.404112,29.530795,30.166599,13.218529,8.938100,828.629153
1,2.0,74.333333,8.0,642.001611,293.921538,264.767499,153.492252,105.143326,100047.019958,16892.070744,...,1.169092,4.117962,6.532768,2.525268,1.426223,28.903283,25.224753,12.442629,9.116965,808.983063
2,3.0,83.000000,8.0,623.870848,291.964848,263.895176,157.205173,100.655201,82353.442963,14736.391053,...,1.138754,4.040329,6.290170,2.441629,1.356562,26.698815,21.851104,10.956077,8.114211,672.190565
3,4.0,85.666667,8.0,510.612076,233.010028,210.645494,123.954499,80.458537,79526.650194,10597.408317,...,1.174022,4.137301,6.858062,2.631444,1.437465,29.956208,28.430755,13.691870,9.468924,832.021445
4,5.0,146.333333,10.0,426.188669,223.960324,207.407505,130.204908,66.522749,43070.950822,6669.229487,...,1.013614,3.286424,5.561892,2.154343,1.245811,19.189221,14.266504,7.558345,5.741588,870.516099
5,6.0,166.666667,10.0,444.097340,234.782142,217.308213,138.078292,69.767190,42249.313234,7139.497653,...,0.987452,3.279710,5.149347,2.033763,1.145403,17.112630,11.779800,6.650701,5.277719,881.490501
6,7.0,214.333333,10.0,296.683127,156.582861,145.243371,90.576198,46.134377,29081.355276,3095.411320,...,1.012850,3.374919,5.403251,2.149089,1.176140,18.805765,12.986054,7.247627,5.646304,1241.747441
7,8.0,203.000000,12.0,708.948804,327.162766,294.613773,173.749242,115.357722,97428.128465,20513.213634,...,1.116399,3.803847,6.773311,2.672986,1.404390,27.737735,27.493027,12.148407,8.218802,1837.532801
8,9.0,199.333333,12.0,802.868097,351.689919,312.281348,175.014586,134.624877,125284.872246,27550.673411,...,1.158854,3.807037,7.248914,2.823805,1.498450,30.254949,31.855411,13.854024,9.532479,2130.831284
9,10.0,159.666667,12.0,367.077671,200.070006,186.465406,119.939012,56.161829,39004.644993,4832.335376,...,1.000518,3.372486,5.349196,2.109050,1.168569,19.182869,14.767607,7.450016,5.722417,925.455507


In [9]:
measurements.to_csv(path + "08-measurements-processed.csv", index=False)